# Initial Recon
This notebook contains commands and procedures for conducting initial reconnaissance on a target. Customize the variables below according to your specific requirements.


## Define Assessment Type


In [ ]:
#Assessment Type
_var_assess_type = ['red_team', 'internal', 'purple_team', 'web_assess']
_var_assess_name = 'TestAssess1'

## Global Variables/Commands

In [ ]:
# Imports
import os
import subprocess
import datetime
from IPython.display import display, Markdown

In [ ]:
# Global Variables
_var_target_ip = '127.0.0.1'
_var_log_location = f'Logs/{_var_assess_name}'


In [ ]:
# Append to Log
if not os.path.exists(_var_log_location):
    open(f'{_var_log_location}.txt', 'a').close()
    
def _func_append_to_log(source, command, output):
    with open(f'{_var_log_location}.txt', 'a') as log_file:
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_file.write(f'Timestamp: {timestamp}\n')
        log_file.write(f'Source: {source}\n')
        log_file.write(f'Command: {command}\n')
        log_file.write(f'Output:\n{output}\n')
        log_file.write('=' * 70 + '\n')


In [ ]:
# Run OS Commands
def command_exe_win(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    if process.returncode == 0:
        #display(Markdown(f"```\n{out.decode('utf-8')}\n```"))
        output = out.decode('utf-8').strip()
        _func_append_to_log('Local', command, output)
        return output
    else:
        error_output = err.decode('utf-8')
        _func_append_to_log('Local', command, error_output)
        return error_output

In [ ]:
#VM Config
_var_vm_host = '192.168.138.130'
_var_vm_port = 22
_var_vm_username = 'bloodvault@kalidev'
_var_vm_password = ''
_var_vm_key = 'Keys/kali_dev'


In [ ]:
# Run VM Commands
import paramiko

def _func_command_exe_vm(command):
    # Initialize the SSH client
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    # Connect to the VM
    if _var_vm_key:
        key = paramiko.RSAKey.from_private_key_file(_var_vm_key, None)
        client.connect(hostname=_var_vm_host, port=_var_vm_port, username=_var_vm_username, pkey=key)
    else:
        client.connect(hostname=_var_vm_host, port=_var_vm_port, username=_var_vm_username, password=_var_vm_password)
    
    # Execute the command
    stdin, stdout, stderr = client.exec_command(command)
    output = stdout.read().decode('utf-8')
    error = stderr.read().decode('utf-8')
    
    # Close the SSH connection
    client.close()
    if error == '':
        _func_append_to_log('VM', command, output)
        return output
    else:
        _func_append_to_log('VM', command, f'Error: {error}')
        return output, error

## Environment Check


In [ ]:
# Find External IP
cell_executed = False
if not cell_executed:
    command_ext_ip = f'curl ip.beer'
    ext_ip = command_exe_win(command_ext_ip)
    print(ext_ip)
    cell_executed = True
else:
    pass

## Scope

In [ ]:
# Alives Check
command_ping = f'ping -n 4 {_var_target_ip}'
ping = command_exe_win(command_ping)
print(ping)

In [ ]:
# DNS Lookup
command_dns_lookup = f'nslookup {ext_ip}'
dns = command_exe_win(command_dns_lookup)
print(dns)

In [ ]:
# Hostname from VM
command_getvmhostname = f'hostname'
_var_vm_hostname = _func_command_exe_vm(command_getvmhostname)
print(_var_vm_hostname)